In [4]:
import psycopg2
import pymongo
import datetime
import base64
from decimal import Decimal
import re

In [5]:
"""
Connection to PostgreSQL DB
"""
conn_string = "host='localhost' dbname='dvdrental' user='postgres'"
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

In [6]:
"""
Base table names retrieving
"""
table_names = []
cursor.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema='public'
    AND table_type='BASE TABLE';
    """)
for table in cursor.fetchall():
    table_names.append(table[0])

In [7]:
"""
Mapping table names to column name collections
"""
table_columns = {}
for name in table_names:
    cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_schema = 'public' AND table_name = '" + name + "';")
    table_columns[name] = []
    for column in cursor.fetchall():
        table_columns[name].append(column[0])
    

In [8]:
"""
Retrieving all objects from selected table
"""
def select_all_from(table, cur):
    cur.execute('SELECT * FROM ' + table + ';')
    return cur.fetchall()

In [9]:
"""
Retrieving all objects from selected table as dict instances with types compatible to MongoDB
"""
def get_dict_list(table, cur):
    columns = table_columns[table]
    res = []
    l = select_all_from(table, cur)
    for rec in l:
        temp = {}
        for i in range(len(columns)):
            if isinstance(rec[i], datetime.date):
                temp[columns[i]] = datetime.datetime.combine(rec[i], datetime.time.min)
            elif isinstance(rec[i], memoryview):
                temp[columns[i]] = str(base64.b64encode(rec[i]))
            elif isinstance(rec[i], Decimal):
                temp[columns[i]] = float(rec[i])
            else:
                temp[columns[i]] = rec[i]
        res.append(temp)
    return res

In [10]:
"""
Dropping previous documents if exist
"""
client = pymongo.MongoClient('localhost', 27017)
db = client.db
for col in db.list_collection_names():
    db[col].drop()

In [11]:
"""
Migrating data from PostgreSQL DB to MongoDB
"""
for table in table_names:
    col = db[table]
    col.insert_many(get_dict_list(table, cursor))

In [12]:
"""
Removing relation table film_actor with saving corresponding relations
"""
for actor in db.actor.find():
    film_ids = []
    for film_bind in db.film_actor.find(
        {'actor_id': actor['actor_id']}
    ):
        film_ids.append(
            db.film.find_one(
                {'film_id': film_bind['film_id']}
            )['_id']
        )
    actor['films'] = film_ids
    db.actor.save(actor)
db.film_actor.drop()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead
  from ipykernel import kernelapp as app


In [13]:
"""
Removing relation table film_category with saving corresponding relations
"""
for film in db.film.find():
    category_id = db.film_category.find_one({'film_id':film['film_id']})['_id']
    film['category'] = category_id
    db.film.save(film)
db.film_category.drop()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead
  import sys


In [14]:
"""
Replacing all related objects' indexes with ObjectIDs
"""
for i in range(2):
    for col in db.list_collection_names():
        docs = db[col].find()
        for doc in docs:
            for key in doc.keys():
                if re.fullmatch('[a-z]+_id$', key) is not None and re.sub('_id$', '', key) != col:
                    col_rel = re.sub('_id$', '', key)
                    id = doc.pop(key)
                    doc[col_rel] = db[col_rel].find_one({key:id})['_id']
            db[col].save(doc)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead
  del sys.path[0]


In [15]:
"""
Removing objects' IDs
"""
for col in db.list_collection_names():
    docs = db[col].find()
    for doc in docs:
        doc.pop(col+'_id')
        db[col].save(doc)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead
  


In [16]:
for doc in db.rental.find():
    inventory = db.inventory.find_one({'_id': doc['inventory']})
    doc['film'] = inventory['film']
    doc['store'] = inventory['store']
    doc.pop('inventory')
    db.rental.save(doc)
db.inventory.drop()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead
  


In [21]:
for col in db.list_collection_names():
    db[col].update_many({}, {'$unset': {'last_update':''}})